In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm

In [2]:
operations = {
    "orf": "wellpos_cc_var_mad_outlier_featselect_sphering_harmony",
    "crispr": "wellpos_cc_var_mad_outlier_featselect_sphering_harmony_PCA_corrected",
}

protein_class_list = [
    "Enzymes",
    "Transporters",
    "G-protein coupled receptors",
    "Transcription factors",
    "Immunoglobulin genes",
    "T-cell receptor genes",
]

variable = "protein_class"
col_name = f"Metadata_{variable}"
col_name_in_metadata = "Metadata_protein_class"
output_df = pd.DataFrame()

In [3]:
# Read phenotypic activity

orf_phenotypic_activity_df = pd.read_csv(
    f"../03.retrieve-annotations/output/phenotypic-activity-{operations['orf']}.csv.gz"
)[["Metadata_JCP2022", "mean_average_precision", "below_corrected_p"]]

crispr_phenotypic_activity_df = pd.read_csv(
    f"../03.retrieve-annotations/output/phenotypic-activity-{operations['crispr']}.csv.gz"
)[["Metadata_JCP2022", "mean_average_precision", "below_corrected_p"]]

In [4]:
# Read metadata

orf_metadata_df = (
    pd.read_csv(
        "../00.download-and-process-annotations/output/orf_metadata.tsv.gz", sep="\t"
    )[["Metadata_JCP2022", col_name_in_metadata]]
    .assign(col=lambda x: x[col_name_in_metadata].str.split("|"))
    .explode("col")
    .query("col in @protein_class_list")
    .drop(columns=[col_name_in_metadata])
    .rename(columns={"col": col_name})
    .drop_duplicates(subset="Metadata_JCP2022", keep="first")
)

crispr_metadata_df = (
    pd.read_csv(
        "../00.download-and-process-annotations/output/crispr_metadata.tsv.gz", sep="\t"
    )[["Metadata_JCP2022", col_name_in_metadata]]
    .assign(col=lambda x: x[col_name_in_metadata].str.split("|"))
    .explode("col")
    .query("col in @protein_class_list")
    .drop(columns=[col_name_in_metadata])
    .rename(columns={"col": col_name})
    .drop_duplicates(subset="Metadata_JCP2022", keep="first")
)

In [5]:
orf_df = orf_phenotypic_activity_df.merge(
    orf_metadata_df, on="Metadata_JCP2022", how="inner"
).dropna(subset=[col_name])
crispr_df = crispr_phenotypic_activity_df.merge(
    crispr_metadata_df, on="Metadata_JCP2022", how="inner"
).dropna(subset=[col_name])

In [6]:
# Create binary column for each protein class
for protein_class in protein_class_list:
    orf_df = orf_df.assign(
        **{
            protein_class: lambda x: x.apply(
                lambda y: True if y[col_name] == protein_class else False, axis=1
            )
        }
    )

    crispr_df = crispr_df.assign(
        **{
            protein_class: lambda x: x.apply(
                lambda y: True if y[col_name] == protein_class else False, axis=1
            )
        }
    )

Fisher's exact test ORF

In [7]:
for protein_class in protein_class_list:
    table = sm.stats.Table.from_data(orf_df[["below_corrected_p", protein_class]])

    odds_ratio, pvalue = stats.fisher_exact(table.table)
    print(f"protein class: {protein_class}, odds ratio: {odds_ratio}, p-value: {pvalue}")

    output_df = pd.concat(
        [
            output_df,
            pd.DataFrame(
                {
                    "protein_class": protein_class,
                    "modality": ["ORF"],
                    f"{variable}_False_phenotype_False": int(table.table[0, 0]),
                    f"{variable}_False_phenotype_True": int(table.table[0, 1]),
                    f"{variable}_True_phenotype_False": int(table.table[1, 0]),
                    f"{variable}_True_phenotype_True": int(table.table[1, 1]),
                    "odds_ratio": odds_ratio,
                    "pvalue": pvalue,
                },
                index=[0],
            ),
        ], 
        ignore_index=True
    )

output_df.query("modality == 'ORF' and pvalue < 0.05")

protein class: Enzymes, odds ratio: 0.9649187017857904, p-value: 0.5267826441272055
protein class: Transporters, odds ratio: 1.092797501190818, p-value: 0.19525544446231224
protein class: G-protein coupled receptors, odds ratio: 1.1039905423663385, p-value: 0.3982211082508736
protein class: Transcription factors, odds ratio: 0.944873617882293, p-value: 0.47144656568073384
protein class: Immunoglobulin genes, odds ratio: 0.7517110645515315, p-value: 0.47047757721439554
protein class: T-cell receptor genes, odds ratio: 0.09464232520532789, p-value: 0.008368021461326286


,protein_class,modality,protein_class_False_phenotype_False,protein_class_False_phenotype_True,protein_class_True_phenotype_False,protein_class_True_phenotype_True,odds_ratio,pvalue
5,T-cell receptor genes,ORF,2224,7,3357,1,0.094642,0.008368


Fisher's exact test CRISPR

In [8]:
for protein_class in protein_class_list:
    table = sm.stats.Table.from_data(crispr_df[["below_corrected_p", protein_class]])
    if table.table.shape != (2, 2):
        continue

    odds_ratio, pvalue = stats.fisher_exact(table.table)
    print(f"protein class: {protein_class}, odds ratio: {odds_ratio}, p-value: {pvalue}")

    output_df = pd.concat(
        [
            output_df,
            pd.DataFrame(
                {
                    "protein_class": protein_class,
                    "modality": ["CRISPR"],
                    f"{variable}_False_phenotype_False": int(table.table[0, 0]),
                    f"{variable}_False_phenotype_True": int(table.table[0, 1]),
                    f"{variable}_True_phenotype_False": int(table.table[1, 0]),
                    f"{variable}_True_phenotype_True": int(table.table[1, 1]),
                    "odds_ratio": odds_ratio,
                    "pvalue": pvalue,
                },
                index=[0],
            ),
        ], 
        ignore_index=True
    )

output_df.query("modality == 'CRISPR' and pvalue < 0.05")

protein class: Enzymes, odds ratio: 1.0680621118012423, p-value: 0.2779923565921355
protein class: Transporters, odds ratio: 0.8815086925444648, p-value: 0.1268978536821487
protein class: G-protein coupled receptors, odds ratio: 0.8552735204692877, p-value: 0.18756251397232349
protein class: Transcription factors, odds ratio: 1.0716392902024046, p-value: 0.38160977847929467


,protein_class,modality,protein_class_False_phenotype_False,protein_class_False_phenotype_True,protein_class_True_phenotype_False,protein_class_True_phenotype_True,odds_ratio,pvalue


In [9]:
output_df.to_csv(f"output/{variable}.csv", index=False)